In [1]:
import pandas as pd
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModel, 
    BertPreTrainedModel, 
    TrainingArguments, 
    Trainer
)
from transformers.modeling_outputs import SequenceClassifierOutput
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import accelerate
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from tqdm.auto import tqdm

tqdm.pandas()

# CUDAのデバッグ用（RuntimeErrorが発生した場合に備えて）
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# --- 1. GPUの確認 ---
if torch.cuda.is_available():
    print(f"✅ GPU is available. Device: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
else:
    print("⚠️ GPU not found. Running on CPU.")
    device = torch.device("cpu")

✅ GPU is available. Device: NVIDIA RTX A6000


In [ ]:
# --- 2. 設定 (本番用) ---

TRAINING_FILE = "data/processed/training_dataset_abstract_cleaned_v3.csv"
MODEL_CHECKPOINT = "allenai/scibert_scivocab_uncased"
OUTPUT_MODEL_DIR = "models/sbert_triplet_v1" # 本番用の保存先

# --- 実験計画からのハイパーパラメータ ---
MAX_LENGTH = 512
LEARNING_RATE = 2e-5
TRIPLET_MARGIN = 1.0
BATCH_SIZE = 32
EPOCHS = 3       # まずは3エポックで実験

print(f"Configuration set for PRODUCTION (Batch Size: {BATCH_SIZE}, Epochs: {EPOCHS})")

Configuration set for PRODUCTION (Batch Size: 64, Epochs: 3)


In [3]:
# --- 3. カスタムモデルクラスの定義 (CLS Pooling) ---

class SiameseTripletModel(BertPreTrainedModel):
    """
    S-BERT (Triplet) モデル (CLS Pooling)
    """
    def __init__(self, config):
        super(SiameseTripletModel, self).__init__(config)
        self.bert = AutoModel.from_config(config)
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        input_ids_positive=None,
        attention_mask_positive=None,
        input_ids_negative=None,
        attention_mask_negative=None,
        labels=None, # TripletLossでは 'labels' は使わない
        **kwargs
    ):
        output_anchor = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output_positive = self.bert(input_ids=input_ids_positive, attention_mask=attention_mask_positive)
        output_negative = self.bert(input_ids=input_ids_negative, attention_mask=attention_mask_negative)

        vec_anchor = output_anchor.pooler_output
        vec_positive = output_positive.pooler_output
        vec_negative = output_negative.pooler_output

        return SequenceClassifierOutput(
            loss=None,
            logits=(vec_anchor, vec_positive, vec_negative),
            hidden_states=None,
            attentions=None,
        )

print("Custom model class 'SiameseTripletModel' (CLS Pooling) defined.")

Custom model class 'SiameseTripletModel' (CLS Pooling) defined.


In [4]:
# --- 4. データセットの読み込みとTripletへの変換 ---
print(f"Loading full dataset: {TRAINING_FILE}")
df_full = pd.read_csv(TRAINING_FILE)
df_full = df_full.dropna(subset=['abstract_a', 'abstract_b', 'label'])
df_full['label'] = df_full['label'].astype(int)
print(f"Full dataset size: {len(df_full)}")

# 全データセットを正例ペアと負例ペアに分割
pos_df = df_full[df_full['label'] == 1]
neg_df = df_full[df_full['label'] == 0]

if pos_df.empty or neg_df.empty:
    raise ValueError("Full dataset must contain both positive and negative samples.")

# --- Tripletの作成 ---
print("Creating triplets from full dataset...")
triplets = []
# 負例のアブストラクトBのリスト（ランダムサンプリング用）
negative_abstracts = neg_df['abstract_b'].tolist()

for index, row in tqdm(pos_df.iterrows(), total=len(pos_df), desc="Creating Triplets"):
    anchor = row['abstract_a']
    positive = row['abstract_b']
    negative = np.random.choice(negative_abstracts)
    
    triplets.append({
        'anchor': anchor,
        'positive': positive,
        'negative': negative
    })

df_triplets = pd.DataFrame(triplets)

# Trainerの評価ループを動作させるため、ダミーのlabels列を追加
df_triplets['labels'] = 0

print(f"Created {len(df_triplets)} triplets.")

# Triplet形式のデータセットをHugging Face形式に変換
raw_dataset = Dataset.from_pandas(df_triplets)
# 本番なので、検証データ(validation)も多め(20%)に確保
dataset_split = raw_dataset.train_test_split(test_size=0.2, seed=42)
dataset = DatasetDict({
    'train': dataset_split['train'],
    'validation': dataset_split['test']
})
print(f"Triplet dataset loaded: {dataset}")

Loading full dataset: data/processed/training_dataset_abstract_cleaned_v3.csv
Full dataset size: 34624
Creating triplets from full dataset...


Creating Triplets:   0%|          | 0/7013 [00:00<?, ?it/s]

Created 7013 triplets.
Triplet dataset loaded: DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'negative', 'labels'],
        num_rows: 5610
    })
    validation: Dataset({
        features: ['anchor', 'positive', 'negative', 'labels'],
        num_rows: 1403
    })
})


In [5]:
# --- 5. トークナイズ処理 (Triplet用) ---
print("Initializing tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def tokenize_triplet_function(examples):
    tokenized_anchor = tokenizer(examples["anchor"], padding="max_length", truncation=True, max_length=MAX_LENGTH)
    tokenized_positive = tokenizer(examples["positive"], padding="max_length", truncation=True, max_length=MAX_LENGTH)
    tokenized_negative = tokenizer(examples["negative"], padding="max_length", truncation=True, max_length=MAX_LENGTH)
    
    return {
        "input_ids": tokenized_anchor["input_ids"],
        "attention_mask": tokenized_anchor["attention_mask"],
        "input_ids_positive": tokenized_positive["input_ids"],
        "attention_mask_positive": tokenized_positive["attention_mask"],
        "input_ids_negative": tokenized_negative["input_ids"],
        "attention_mask_negative": tokenized_negative["attention_mask"],
    }

print("Tokenizing triplet dataset...")
tokenized_datasets = dataset.map(tokenize_triplet_function, batched=True, num_proc=4,
                                 remove_columns=["anchor", "positive", "negative"])
tokenized_datasets.set_format("torch")
print("Tokenization complete.")

Initializing tokenizer...


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Tokenizing triplet dataset...


Map (num_proc=4):   0%|          | 0/5610 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1403 [00:00<?, ? examples/s]

Tokenization complete.


In [6]:
# --- 6. カスタムTrainerの定義 ---

class TripletTrainer(Trainer):
    def __init__(self, *args, margin=1.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.margin = margin
        self.loss_fct = nn.TripletMarginLoss(margin=self.margin)
        print(f"TripletTrainer initialized with margin={self.margin}")

    def compute_loss(self, model, inputs, return_outputs=False):
        if "labels" in inputs:
            inputs.pop("labels")
            
        outputs = model(**inputs)
        vec_anchor, vec_positive, vec_negative = outputs.logits
        loss = self.loss_fct(vec_anchor, vec_positive, vec_negative)

        return (loss, outputs) if return_outputs else loss

print("Custom 'TripletTrainer' defined.")

Custom 'TripletTrainer' defined.


In [7]:
# --- 7. モデルのロードと訓練設定 ---
print(f"Loading custom model: {MODEL_CHECKPOINT}")
model = SiameseTripletModel.from_pretrained(MODEL_CHECKPOINT).to(device)
print("Custom model loaded.")

# 評価指標を計算する関数 (Triplet Accuracy)
def compute_metrics_triplet(eval_pred):
    vec_a, vec_p, vec_n = eval_pred.predictions
    dist_pos = F.pairwise_distance(torch.tensor(vec_a), torch.tensor(vec_p))
    dist_neg = F.pairwise_distance(torch.tensor(vec_a), torch.tensor(vec_n))
    accuracy = (dist_pos < dist_neg).float().mean().item()
    return {'triplet_accuracy': accuracy}

# 訓練の設定
training_args = TrainingArguments(
    output_dir=OUTPUT_MODEL_DIR,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE, # 64
    per_device_eval_batch_size=BATCH_SIZE,  # 64
    num_train_epochs=EPOCHS,                # 3
    weight_decay=0.01,
    evaluation_strategy="epoch", # エポックごとに評価
    save_strategy="epoch",       # エポックごとに保存
    load_best_model_at_end=True, # 最高のモデルを最後にロード
    logging_strategy="steps",
    logging_steps=100,           # 100ステップごとにログ表示
    warmup_ratio=0.1,            # 10%ウォームアップ
    bf16=True,
)
print("Training arguments set for PRODUCTION.")

Loading custom model: allenai/scibert_scivocab_uncased
Custom model loaded.
Training arguments set for PRODUCTION.


In [8]:
# --- 8. 訓練の開始 ---
trainer = TripletTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_triplet,
    margin=TRIPLET_MARGIN
)

print("\n--- Starting Model Training (Triplet Loss PRODUCTION) ---")
trainer.train()
print("--- Model Training Complete ---")

TripletTrainer initialized with margin=1.0

--- Starting Model Training (Triplet Loss PRODUCTION) ---


OutOfMemoryError: CUDA out of memory. Tried to allocate 768.00 MiB. GPU 

In [ ]:
# --- 9. モデルの保存 ---
print("Training complete. Saving best model...")
best_model_path = os.path.join(OUTPUT_MODEL_DIR, "best_model")
trainer.save_model(best_model_path)
print(f"Model saved to {best_model_path}")

In [ ]:
# --- 10. 訓練結果の可視化 ---
print("\n--- Visualizing Training Results ---")
log_history = trainer.state.log_history
df_log = pd.DataFrame(log_history)

df_train = df_log[df_log['loss'].notna()].copy()
df_eval = df_log[df_log['eval_loss'].notna()].copy()

# 'epoch'列を整数型に（表示のため）
if 'epoch' in df_train.columns:
    df_train['epoch'] = df_train['epoch'].astype(int)
if 'epoch' in df_eval.columns:
    df_eval['epoch'] = df_eval['epoch'].astype(int)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
plt.style.use('seaborn-v0_8-whitegrid')

if not df_train.empty:
    sns.lineplot(data=df_train, x='epoch', y='loss', label='Training Loss', ax=ax1, marker='o')
if not df_eval.empty:
    sns.lineplot(data=df_eval, x='epoch', y='eval_loss', label='Validation Loss', ax=ax1, marker='o')
ax1.set_title('Training vs. Validation Loss')
ax1.set_ylabel('Loss')
ax1.set_xlabel('Epoch') # X軸をEpochに
ax1.legend()

if not df_eval.empty:
    sns.lineplot(data=df_eval, x='epoch', y='eval_triplet_accuracy', label='Validation Triplet Accuracy', ax=ax2, marker='o')
ax2.set_title('Validation Metrics')
ax2.set_ylabel('Accuracy')
ax2.set_xlabel('Epoch') # X軸をEpochに
ax2.legend()
ax2.set_ylim(0, 1)

plt.tight_layout()
plt.show()

print("\n--- Best Model Evaluation Metrics (from validation set) ---")
if not df_eval.empty:
    best_run = df_eval.loc[df_eval['eval_loss'].idxmin()]
    print(f"Best Epoch (based on min eval_loss): {best_run['epoch']}")
    print(f"Best Validation Loss: {best_run['eval_loss']:.4f}")
    print(f"Best Validation Triplet Accuracy: {best_run['eval_triplet_accuracy']:.4f}")
else:
    print("No evaluation steps were completed.")